## Data Cleaning

Tutorial on how to clean data with numpy and pandas

Sources:
- [1] https://realpython.com/python-data-cleaning-numpy-pandas/

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load data from csv file
df = pd.read_csv('BL-Flickr-Images-Book.csv')
df.head() # First 5 entries of the dataframe

In [ ]:
# We don't want all of these columns, so lets drop some of them
to_drop = ['Edition Statement','Corporate Author','Corporate Contributors','Former owner','Engraver','Contributors','Issuance type','Shelfmarks']

# inplace means modifying the object directly, axis denotes where the values to be dropped are
df.drop(to_drop, inplace=True, axis=1)

# Alternitavely we could have done
# df.drop(columns=to_drop, inplace=True)
# Or even read from the csv with specific columns
# df = pd.read_csv('FILENAME', usecols=to_drop)

df.head()

In [ ]:
# The index of a dataframe should be a uniquely valued identifying field
df['Identifier'].is_unique

In [ ]:
# Lets use this column as our index
df = df.set_index('Identifier') # Note we could have used the inplace here to modify original object
df.head()

In [ ]:
# We can now use this index to slice the dataframe
df.loc[206]

In [ ]:
# How many different data types are there in this dataframe
df.get_dtype_counts()

In [ ]:
# We can see that the data column is noisy
df.loc[1905:, 'Date of Publication'].head(10)

In [ ]:
# Let's clean it up with a regex
regex = r'^(\d{4})' # This finds any 4 digits in the beginning of a string

extr = df['Date of Publication'].str.extract(regex, expand=False)
extr.head()

In [ ]:
# This has the object data type though
extr.get_dtype_counts()

In [ ]:
# Lets make it numeric
df['Date of Publication'] = pd.to_numeric(extr)
df['Date of Publication'].dtype

In [ ]:
# How many of the values are missing though?
df['Date of Publication'].isnull().sum() / len(df)

In [ ]:
# Lets look at a different column we need to clean up
df['Place of Publication'].head(10)

In [ ]:
# Its difficult to write a regex to clean this up, so lets instead
# use the np.where() function
pub = df['Place of Publication']

 # boolean mask for containing the strings London and Oxford
london = pub.str.contains('London')
oxford = pub.str.contains('Oxford')

# Replace using nested np.where
df['Place of Publication'] = np.where(london, 'London',
                                     np.where(oxford, 'Oxford',
                                             pub.str.replace('-', ' ')))

df['Place of Publication'].head()

In [ ]:
# If we now look at the top entries they look mcuh cleaner
df.head()

# applymap()

It is often easier to apply a customized function to each cell or element directly with a function such as applymap()

In [ ]:
university_towns = []
with open('university_towns.txt') as file:
    for line in file:
        if '[edit]' in line:
            state = line # Remember this state
        else:
            university_towns.append((state, line))

university_towns[:5]

In [ ]:
# Feed this list of tuples into a dataframe
towns_df = pd.DataFrame(university_towns,
                       columns=['State', 'RegionName'])

towns_df.head()

In [ ]:
# Define a function to apply to each cell
def get_citystate(item):
    if ' (' in item:
        return item[:item.find(' (')]
    if '[' in item:
        return item[:item.find('[')]
    else:
        return item
    
towns_df = towns_df.applymap(get_citystate)
towns_df.head()